In [ ]:
# only need to run this the first time you use the notebook
! pip install -U -q --user google-cloud-vision
# must restart kernel after running this

In [1]:
# imports
from google.cloud import vision
from PIL import Image, ImageDraw, ImageFont

In [6]:
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    
    items = list()

    objects = client.object_localization(image=image).localized_object_annotations

    print(f"Number of objects found: {len(objects)}")
    for object_ in objects:
        print(f"\n{object_.name} (confidence: {object_.score})")
        v = list()
        for vertex in object_.bounding_poly.normalized_vertices:
            v.append({"x": vertex.x, "y": vertex.y})
            print(f" - ({vertex.x}, {vertex.y})")
        items.append({"label": object_.name, "vertices": v})
    return items
    
def draw_bounding_boxes(image_path, bounding_boxes):
    """
    Draws bounding boxes with labels on an image.
    
    Args:
    - image_path: Path to the image file.
    - bounding_boxes: List of dictionaries containing 'label' and 'vertices'.
                      The 'vertices' should be normalized coordinates (0 to 1).
    """
    # Open the image file
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # Image dimensions
    img_width, img_height = image.size
    
    # Optional: Load a font (you can use a default font if PIL doesn't have a custom font)
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()
    
    # Loop through bounding boxes
    for box in bounding_boxes:
        label = box['label']
        vertices = box['vertices']
        
        # Convert normalized coordinates to actual image coordinates
        points = [(v['x'] * img_width, v['y'] * img_height) for v in vertices]
        
        # Draw the bounding box
        draw.polygon(points, outline="red", width=3)
        
        # Get the top-left point for the label
        top_left = points[0]
        
        # Draw the label in the top-left corner of the bounding box
        draw.text(top_left, label, fill="red", font=font)
    
    # Save or show the modified image
    output_file = "output.png"
    image.save(output_file) #to save it instead of showing it
    print(f"Image save to: {output_file}")

In [7]:
image_path = "pic_of_items.png"
bounding_boxes = localize_objects(image_path)
draw_bounding_boxes(image_path, bounding_boxes)

Number of objects found: 3

Hat (confidence: 0.942505955696106)
 - (0.53515625, 0.1455078125)
 - (0.98046875, 0.1455078125)
 - (0.98046875, 0.51171875)
 - (0.53515625, 0.51171875)

Top (confidence: 0.8679339289665222)
 - (0.2333984375, 0.1494140625)
 - (0.43359375, 0.1494140625)
 - (0.43359375, 0.62109375)
 - (0.2333984375, 0.62109375)

Top (confidence: 0.7445529103279114)
 - (0.078125, 0.04345703125)
 - (0.271484375, 0.04345703125)
 - (0.271484375, 0.5078125)
 - (0.078125, 0.5078125)
Image save to: output.png
